In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit
from simtk import openmm
import argparse
import os
import time
from simtk.openmm.app import PDBFile
import mdtraj as md

# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.DEBUG)

In [46]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

# Define simulation parameters
nsteps_eq = 2
nsteps_neq = 1
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
temperature = 300 * unit.kelvin



In [26]:
# Read in htf
with open("/data/chodera/zhangi/perses_benchmark/neq/8/31/31_apo.pickle", 'rb') as f:
    htf = pickle.load(f)
system = htf.hybrid_system
positions = htf.hybrid_positions


In [31]:
# Set solute atom masses to be 0, unless they are part of the residue that's being mutated
for atom in htf.hybrid_topology.atoms:
    if atom.residue.name not in ('WAT', 'HOH', 'NA', 'CL'):
        if atom.residue.index != 42 and atom.residue.chain.index == 0:
            system.setParticleMass(atom.index, 0.0)


In [32]:
for atom in htf.hybrid_topology.atoms:
    print(atom.index, system.getParticleMass(int(atom.index)))


0 0.0 Da
1 0.0 Da
2 0.0 Da
3 0.0 Da
4 0.0 Da
5 0.0 Da
6 0.0 Da
7 0.0 Da
8 0.0 Da
9 0.0 Da
10 0.0 Da
11 0.0 Da
12 0.0 Da
13 0.0 Da
14 0.0 Da
15 0.0 Da
16 0.0 Da
17 0.0 Da
18 0.0 Da
19 0.0 Da
20 0.0 Da
21 0.0 Da
22 0.0 Da
23 0.0 Da
24 0.0 Da
25 0.0 Da
26 0.0 Da
27 0.0 Da
28 0.0 Da
29 0.0 Da
30 0.0 Da
31 0.0 Da
32 0.0 Da
33 0.0 Da
34 0.0 Da
35 0.0 Da
36 0.0 Da
37 0.0 Da
38 0.0 Da
39 0.0 Da
40 0.0 Da
41 0.0 Da
42 0.0 Da
43 0.0 Da
44 0.0 Da
45 0.0 Da
46 0.0 Da
47 0.0 Da
48 0.0 Da
49 0.0 Da
50 0.0 Da
51 0.0 Da
52 0.0 Da
53 0.0 Da
54 0.0 Da
55 0.0 Da
56 0.0 Da
57 0.0 Da
58 0.0 Da
59 0.0 Da
60 0.0 Da
61 0.0 Da
62 0.0 Da
63 0.0 Da
64 0.0 Da
65 0.0 Da
66 0.0 Da
67 0.0 Da
68 0.0 Da
69 0.0 Da
70 0.0 Da
71 0.0 Da
72 0.0 Da
73 0.0 Da
74 0.0 Da
75 0.0 Da
76 0.0 Da
77 0.0 Da
78 0.0 Da
79 0.0 Da
80 0.0 Da
81 0.0 Da
82 0.0 Da
83 0.0 Da
84 0.0 Da
85 0.0 Da
86 0.0 Da
87 0.0 Da
88 0.0 Da
89 0.0 Da
90 0.0 Da
91 0.0 Da
92 0.0 Da
93 0.0 Da
94 0.0 Da
95 0.0 Da
96 0.0 Da
97 0.0 Da
98 0.0 Da
99 0.0 Da
100 0.0 Da

1084 0.0 Da
1085 0.0 Da
1086 0.0 Da
1087 0.0 Da
1088 0.0 Da
1089 0.0 Da
1090 0.0 Da
1091 0.0 Da
1092 0.0 Da
1093 0.0 Da
1094 0.0 Da
1095 0.0 Da
1096 0.0 Da
1097 0.0 Da
1098 0.0 Da
1099 0.0 Da
1100 0.0 Da
1101 0.0 Da
1102 0.0 Da
1103 0.0 Da
1104 0.0 Da
1105 0.0 Da
1106 0.0 Da
1107 0.0 Da
1108 0.0 Da
1109 0.0 Da
1110 0.0 Da
1111 0.0 Da
1112 0.0 Da
1113 0.0 Da
1114 0.0 Da
1115 0.0 Da
1116 0.0 Da
1117 0.0 Da
1118 0.0 Da
1119 0.0 Da
1120 0.0 Da
1121 0.0 Da
1122 0.0 Da
1123 0.0 Da
1124 0.0 Da
1125 0.0 Da
1126 0.0 Da
1127 0.0 Da
1128 0.0 Da
1129 0.0 Da
1130 0.0 Da
1131 0.0 Da
1132 0.0 Da
1133 0.0 Da
1134 0.0 Da
1135 0.0 Da
1136 0.0 Da
1137 0.0 Da
1138 0.0 Da
1139 0.0 Da
1140 0.0 Da
1141 0.0 Da
1142 0.0 Da
1143 0.0 Da
1144 0.0 Da
1145 0.0 Da
1146 0.0 Da
1147 0.0 Da
1148 0.0 Da
1149 0.0 Da
1150 0.0 Da
1151 0.0 Da
1152 0.0 Da
1153 0.0 Da
1154 0.0 Da
1155 0.0 Da
1156 0.0 Da
1157 0.0 Da
1158 0.0 Da
1159 0.0 Da
1160 0.0 Da
1161 0.0 Da
1162 0.0 Da
1163 0.0 Da
1164 0.0 Da
1165 0.0 Da
1166 0.0 Da
1167

2084 4.0 Da
2085 4.0 Da
2086 10.015324 Da
2087 4.0 Da
2088 4.0 Da
2089 10.015324 Da
2090 4.0 Da
2091 4.0 Da
2092 10.015324 Da
2093 4.0 Da
2094 4.0 Da
2095 10.015324 Da
2096 4.0 Da
2097 4.0 Da
2098 10.015324 Da
2099 4.0 Da
2100 4.0 Da
2101 10.015324 Da
2102 4.0 Da
2103 4.0 Da
2104 10.015324 Da
2105 4.0 Da
2106 4.0 Da
2107 10.015324 Da
2108 4.0 Da
2109 4.0 Da
2110 10.015324 Da
2111 4.0 Da
2112 4.0 Da
2113 10.015324 Da
2114 4.0 Da
2115 4.0 Da
2116 10.015324 Da
2117 4.0 Da
2118 4.0 Da
2119 10.015324 Da
2120 4.0 Da
2121 4.0 Da
2122 10.015324 Da
2123 4.0 Da
2124 4.0 Da
2125 10.015324 Da
2126 4.0 Da
2127 4.0 Da
2128 10.015324 Da
2129 4.0 Da
2130 4.0 Da
2131 10.015324 Da
2132 4.0 Da
2133 4.0 Da
2134 10.015324 Da
2135 4.0 Da
2136 4.0 Da
2137 10.015324 Da
2138 4.0 Da
2139 4.0 Da
2140 10.015324 Da
2141 4.0 Da
2142 4.0 Da
2143 10.015324 Da
2144 4.0 Da
2145 4.0 Da
2146 10.015324 Da
2147 4.0 Da
2148 4.0 Da
2149 10.015324 Da
2150 4.0 Da
2151 4.0 Da
2152 10.015324 Da
2153 4.0 Da
2154 4.0 Da
2155 10.01

3334 10.015324 Da
3335 4.0 Da
3336 4.0 Da
3337 10.015324 Da
3338 4.0 Da
3339 4.0 Da
3340 10.015324 Da
3341 4.0 Da
3342 4.0 Da
3343 10.015324 Da
3344 4.0 Da
3345 4.0 Da
3346 10.015324 Da
3347 4.0 Da
3348 4.0 Da
3349 10.015324 Da
3350 4.0 Da
3351 4.0 Da
3352 10.015324 Da
3353 4.0 Da
3354 4.0 Da
3355 10.015324 Da
3356 4.0 Da
3357 4.0 Da
3358 10.015324 Da
3359 4.0 Da
3360 4.0 Da
3361 10.015324 Da
3362 4.0 Da
3363 4.0 Da
3364 10.015324 Da
3365 4.0 Da
3366 4.0 Da
3367 10.015324 Da
3368 4.0 Da
3369 4.0 Da
3370 10.015324 Da
3371 4.0 Da
3372 4.0 Da
3373 10.015324 Da
3374 4.0 Da
3375 4.0 Da
3376 10.015324 Da
3377 4.0 Da
3378 4.0 Da
3379 10.015324 Da
3380 4.0 Da
3381 4.0 Da
3382 10.015324 Da
3383 4.0 Da
3384 4.0 Da
3385 10.015324 Da
3386 4.0 Da
3387 4.0 Da
3388 10.015324 Da
3389 4.0 Da
3390 4.0 Da
3391 10.015324 Da
3392 4.0 Da
3393 4.0 Da
3394 10.015324 Da
3395 4.0 Da
3396 4.0 Da
3397 10.015324 Da
3398 4.0 Da
3399 4.0 Da
3400 10.015324 Da
3401 4.0 Da
3402 4.0 Da
3403 10.015324 Da
3404 4.0 Da
3405

4584 4.0 Da
4585 10.015324 Da
4586 4.0 Da
4587 4.0 Da
4588 10.015324 Da
4589 4.0 Da
4590 4.0 Da
4591 10.015324 Da
4592 4.0 Da
4593 4.0 Da
4594 10.015324 Da
4595 4.0 Da
4596 4.0 Da
4597 10.015324 Da
4598 4.0 Da
4599 4.0 Da
4600 10.015324 Da
4601 4.0 Da
4602 4.0 Da
4603 10.015324 Da
4604 4.0 Da
4605 4.0 Da
4606 10.015324 Da
4607 4.0 Da
4608 4.0 Da
4609 10.015324 Da
4610 4.0 Da
4611 4.0 Da
4612 10.015324 Da
4613 4.0 Da
4614 4.0 Da
4615 10.015324 Da
4616 4.0 Da
4617 4.0 Da
4618 10.015324 Da
4619 4.0 Da
4620 4.0 Da
4621 10.015324 Da
4622 4.0 Da
4623 4.0 Da
4624 10.015324 Da
4625 4.0 Da
4626 4.0 Da
4627 10.015324 Da
4628 4.0 Da
4629 4.0 Da
4630 10.015324 Da
4631 4.0 Da
4632 4.0 Da
4633 10.015324 Da
4634 4.0 Da
4635 4.0 Da
4636 10.015324 Da
4637 4.0 Da
4638 4.0 Da
4639 10.015324 Da
4640 4.0 Da
4641 4.0 Da
4642 10.015324 Da
4643 4.0 Da
4644 4.0 Da
4645 10.015324 Da
4646 4.0 Da
4647 4.0 Da
4648 10.015324 Da
4649 4.0 Da
4650 4.0 Da
4651 10.015324 Da
4652 4.0 Da
4653 4.0 Da
4654 10.015324 Da
4655

...

9583 10.015324 Da
9584 4.0 Da
9585 4.0 Da
9586 10.015324 Da
9587 4.0 Da
9588 4.0 Da
9589 10.015324 Da
9590 4.0 Da
9591 4.0 Da
9592 10.015324 Da
9593 4.0 Da
9594 4.0 Da
9595 10.015324 Da
9596 4.0 Da
9597 4.0 Da
9598 10.015324 Da
9599 4.0 Da
9600 4.0 Da
9601 10.015324 Da
9602 4.0 Da
9603 4.0 Da
9604 10.015324 Da
9605 4.0 Da
9606 4.0 Da
9607 10.015324 Da
9608 4.0 Da
9609 4.0 Da
9610 10.015324 Da
9611 4.0 Da
9612 4.0 Da
9613 10.015324 Da
9614 4.0 Da
9615 4.0 Da
9616 10.015324 Da
9617 4.0 Da
9618 4.0 Da
9619 10.015324 Da
9620 4.0 Da
9621 4.0 Da
9622 10.015324 Da
9623 4.0 Da
9624 4.0 Da
9625 10.015324 Da
9626 4.0 Da
9627 4.0 Da
9628 10.015324 Da
9629 4.0 Da
9630 4.0 Da
9631 10.015324 Da
9632 4.0 Da
9633 4.0 Da
9634 10.015324 Da
9635 4.0 Da
9636 4.0 Da
9637 10.015324 Da
9638 4.0 Da
9639 4.0 Da
9640 10.015324 Da
9641 4.0 Da
9642 4.0 Da
9643 10.015324 Da
9644 4.0 Da
9645 4.0 Da
9646 10.015324 Da
9647 4.0 Da
9648 4.0 Da
9649 10.015324 Da
9650 4.0 Da
9651 4.0 Da
9652 10.015324 Da
9653 4.0 Da
9654

10832 4.0 Da
10833 4.0 Da
10834 10.015324 Da
10835 4.0 Da
10836 4.0 Da
10837 10.015324 Da
10838 4.0 Da
10839 4.0 Da
10840 10.015324 Da
10841 4.0 Da
10842 4.0 Da
10843 10.015324 Da
10844 4.0 Da
10845 4.0 Da
10846 10.015324 Da
10847 4.0 Da
10848 4.0 Da
10849 10.015324 Da
10850 4.0 Da
10851 4.0 Da
10852 10.015324 Da
10853 4.0 Da
10854 4.0 Da
10855 10.015324 Da
10856 4.0 Da
10857 4.0 Da
10858 10.015324 Da
10859 4.0 Da
10860 4.0 Da
10861 10.015324 Da
10862 4.0 Da
10863 4.0 Da
10864 10.015324 Da
10865 4.0 Da
10866 4.0 Da
10867 10.015324 Da
10868 4.0 Da
10869 4.0 Da
10870 10.015324 Da
10871 4.0 Da
10872 4.0 Da
10873 10.015324 Da
10874 4.0 Da
10875 4.0 Da
10876 10.015324 Da
10877 4.0 Da
10878 4.0 Da
10879 10.015324 Da
10880 4.0 Da
10881 4.0 Da
10882 10.015324 Da
10883 4.0 Da
10884 4.0 Da
10885 10.015324 Da
10886 4.0 Da
10887 4.0 Da
10888 10.015324 Da
10889 4.0 Da
10890 4.0 Da
10891 10.015324 Da
10892 4.0 Da
10893 4.0 Da
10894 10.015324 Da
10895 4.0 Da
10896 4.0 Da
10897 10.015324 Da
10898 4.0 

12082 10.015324 Da
12083 4.0 Da
12084 4.0 Da
12085 10.015324 Da
12086 4.0 Da
12087 4.0 Da
12088 10.015324 Da
12089 4.0 Da
12090 4.0 Da
12091 10.015324 Da
12092 4.0 Da
12093 4.0 Da
12094 10.015324 Da
12095 4.0 Da
12096 4.0 Da
12097 10.015324 Da
12098 4.0 Da
12099 4.0 Da
12100 10.015324 Da
12101 4.0 Da
12102 4.0 Da
12103 10.015324 Da
12104 4.0 Da
12105 4.0 Da
12106 10.015324 Da
12107 4.0 Da
12108 4.0 Da
12109 10.015324 Da
12110 4.0 Da
12111 4.0 Da
12112 10.015324 Da
12113 4.0 Da
12114 4.0 Da
12115 10.015324 Da
12116 4.0 Da
12117 4.0 Da
12118 10.015324 Da
12119 4.0 Da
12120 4.0 Da
12121 10.015324 Da
12122 4.0 Da
12123 4.0 Da
12124 10.015324 Da
12125 4.0 Da
12126 4.0 Da
12127 10.015324 Da
12128 4.0 Da
12129 4.0 Da
12130 10.015324 Da
12131 4.0 Da
12132 4.0 Da
12133 10.015324 Da
12134 4.0 Da
12135 4.0 Da
12136 10.015324 Da
12137 4.0 Da
12138 4.0 Da
12139 10.015324 Da
12140 4.0 Da
12141 4.0 Da
12142 10.015324 Da
12143 4.0 Da
12144 4.0 Da
12145 10.015324 Da
12146 4.0 Da
12147 4.0 Da
12148 10.0

13332 4.0 Da
13333 10.015324 Da
13334 4.0 Da
13335 4.0 Da
13336 10.015324 Da
13337 4.0 Da
13338 4.0 Da
13339 10.015324 Da
13340 4.0 Da
13341 4.0 Da
13342 10.015324 Da
13343 4.0 Da
13344 4.0 Da
13345 10.015324 Da
13346 4.0 Da
13347 4.0 Da
13348 10.015324 Da
13349 4.0 Da
13350 4.0 Da
13351 10.015324 Da
13352 4.0 Da
13353 4.0 Da
13354 10.015324 Da
13355 4.0 Da
13356 4.0 Da
13357 10.015324 Da
13358 4.0 Da
13359 4.0 Da
13360 10.015324 Da
13361 4.0 Da
13362 4.0 Da
13363 10.015324 Da
13364 4.0 Da
13365 4.0 Da
13366 10.015324 Da
13367 4.0 Da
13368 4.0 Da
13369 10.015324 Da
13370 4.0 Da
13371 4.0 Da
13372 10.015324 Da
13373 4.0 Da
13374 4.0 Da
13375 10.015324 Da
13376 4.0 Da
13377 4.0 Da
13378 10.015324 Da
13379 4.0 Da
13380 4.0 Da
13381 10.015324 Da
13382 4.0 Da
13383 4.0 Da
13384 10.015324 Da
13385 4.0 Da
13386 4.0 Da
13387 10.015324 Da
13388 4.0 Da
13389 4.0 Da
13390 10.015324 Da
13391 4.0 Da
13392 4.0 Da
13393 10.015324 Da
13394 4.0 Da
13395 4.0 Da
13396 10.015324 Da
13397 4.0 Da
13398 4.0 

14582 4.0 Da
14583 4.0 Da
14584 10.015324 Da
14585 4.0 Da
14586 4.0 Da
14587 10.015324 Da
14588 4.0 Da
14589 4.0 Da
14590 10.015324 Da
14591 4.0 Da
14592 4.0 Da
14593 10.015324 Da
14594 4.0 Da
14595 4.0 Da
14596 10.015324 Da
14597 4.0 Da
14598 4.0 Da
14599 10.015324 Da
14600 4.0 Da
14601 4.0 Da
14602 10.015324 Da
14603 4.0 Da
14604 4.0 Da
14605 10.015324 Da
14606 4.0 Da
14607 4.0 Da
14608 10.015324 Da
14609 4.0 Da
14610 4.0 Da
14611 10.015324 Da
14612 4.0 Da
14613 4.0 Da
14614 10.015324 Da
14615 4.0 Da
14616 4.0 Da
14617 10.015324 Da
14618 4.0 Da
14619 4.0 Da
14620 10.015324 Da
14621 4.0 Da
14622 4.0 Da
14623 10.015324 Da
14624 4.0 Da
14625 4.0 Da
14626 10.015324 Da
14627 4.0 Da
14628 4.0 Da
14629 10.015324 Da
14630 4.0 Da
14631 4.0 Da
14632 10.015324 Da
14633 4.0 Da
14634 4.0 Da
14635 10.015324 Da
14636 4.0 Da
14637 4.0 Da
14638 10.015324 Da
14639 4.0 Da
14640 4.0 Da
14641 10.015324 Da
14642 4.0 Da
14643 4.0 Da
14644 10.015324 Da
14645 4.0 Da
14646 4.0 Da
14647 10.015324 Da
14648 4.0 

In [13]:
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom.residue.name, atom)

0 ACE ACE0-CH3
1 ACE ACE0-C
2 ACE ACE0-O
3 ACE ACE0-H1
4 ACE ACE0-H2
5 ACE ACE0-H3
6 LYS LYS1-N
7 LYS LYS1-CA
8 LYS LYS1-C
9 LYS LYS1-O
10 LYS LYS1-CB
11 LYS LYS1-CG
12 LYS LYS1-CD
13 LYS LYS1-CE
14 LYS LYS1-NZ
15 LYS LYS1-H2
16 LYS LYS1-HA
17 LYS LYS1-HB3
18 LYS LYS1-HB2
19 LYS LYS1-HG2
20 LYS LYS1-HG3
21 LYS LYS1-HD2
22 LYS LYS1-HD3
23 LYS LYS1-HE3
24 LYS LYS1-HE2
25 LYS LYS1-HZ1
26 LYS LYS1-HZ2
27 LYS LYS1-HZ3
28 LYS LYS2-N
29 LYS LYS2-CA
30 LYS LYS2-C
31 LYS LYS2-O
32 LYS LYS2-CB
33 LYS LYS2-CG
34 LYS LYS2-CD
35 LYS LYS2-CE
36 LYS LYS2-NZ
37 LYS LYS2-H
38 LYS LYS2-HA
39 LYS LYS2-HB3
40 LYS LYS2-HB2
41 LYS LYS2-HG2
42 LYS LYS2-HG3
43 LYS LYS2-HD3
44 LYS LYS2-HD2
45 LYS LYS2-HE3
46 LYS LYS2-HE2
47 LYS LYS2-HZ1
48 LYS LYS2-HZ2
49 LYS LYS2-HZ3
50 ALA ALA3-N
51 ALA ALA3-CA
52 ALA ALA3-C
53 ALA ALA3-O
54 ALA ALA3-CB
55 ALA ALA3-H
56 ALA ALA3-HA
57 ALA ALA3-HB1
58 ALA ALA3-HB2
59 ALA ALA3-HB3
60 VAL VAL4-N
61 VAL VAL4-CA
62 VAL VAL4-C
63 VAL VAL4-O
64 VAL VAL4-CB
65 VAL VAL4-CG1
66 VAL VA

803 VAL VAL50-CB
804 VAL VAL50-CG1
805 VAL VAL50-CG2
806 VAL VAL50-H
807 VAL VAL50-HA
808 VAL VAL50-HB
809 VAL VAL50-HG11
810 VAL VAL50-HG12
811 VAL VAL50-HG13
812 VAL VAL50-HG21
813 VAL VAL50-HG22
814 VAL VAL50-HG23
815 LEU LEU51-N
816 LEU LEU51-CA
817 LEU LEU51-C
818 LEU LEU51-O
819 LEU LEU51-CB
820 LEU LEU51-CG
821 LEU LEU51-CD1
822 LEU LEU51-CD2
823 LEU LEU51-H
824 LEU LEU51-HA
825 LEU LEU51-HB3
826 LEU LEU51-HB2
827 LEU LEU51-HG
828 LEU LEU51-HD11
829 LEU LEU51-HD12
830 LEU LEU51-HD13
831 LEU LEU51-HD21
832 LEU LEU51-HD22
833 LEU LEU51-HD23
834 GLU GLU52-N
835 GLU GLU52-CA
836 GLU GLU52-C
837 GLU GLU52-O
838 GLU GLU52-CB
839 GLU GLU52-CG
840 GLU GLU52-CD
841 GLU GLU52-OE1
842 GLU GLU52-OE2
843 GLU GLU52-H
844 GLU GLU52-HA
845 GLU GLU52-HB3
846 GLU GLU52-HB2
847 GLU GLU52-HG3
848 GLU GLU52-HG2
849 TRP TRP53-N
850 TRP TRP53-CA
851 TRP TRP53-C
852 TRP TRP53-O
853 TRP TRP53-CB
854 TRP TRP53-CG
855 TRP TRP53-CD1
856 TRP TRP53-CD2
857 TRP TRP53-NE1
858 TRP TRP53-CE2
859 TRP TRP53-CE3
86

1637 HOH HOH157-H1
1638 HOH HOH157-H2
1639 HOH HOH158-O
1640 HOH HOH158-H1
1641 HOH HOH158-H2
1642 HOH HOH159-O
1643 HOH HOH159-H1
1644 HOH HOH159-H2
1645 HOH HOH160-O
1646 HOH HOH160-H1
1647 HOH HOH160-H2
1648 HOH HOH161-O
1649 HOH HOH161-H1
1650 HOH HOH161-H2
1651 HOH HOH162-O
1652 HOH HOH162-H1
1653 HOH HOH162-H2
1654 HOH HOH163-O
1655 HOH HOH163-H1
1656 HOH HOH163-H2
1657 HOH HOH164-O
1658 HOH HOH164-H1
1659 HOH HOH164-H2
1660 HOH HOH165-O
1661 HOH HOH165-H1
1662 HOH HOH165-H2
1663 HOH HOH166-O
1664 HOH HOH166-H1
1665 HOH HOH166-H2
1666 HOH HOH167-O
1667 HOH HOH167-H1
1668 HOH HOH167-H2
1669 HOH HOH168-O
1670 HOH HOH168-H1
1671 HOH HOH168-H2
1672 HOH HOH169-O
1673 HOH HOH169-H1
1674 HOH HOH169-H2
1675 HOH HOH170-O
1676 HOH HOH170-H1
1677 HOH HOH170-H2
1678 HOH HOH171-O
1679 HOH HOH171-H1
1680 HOH HOH171-H2
1681 HOH HOH172-O
1682 HOH HOH172-H1
1683 HOH HOH172-H2
1684 HOH HOH173-O
1685 HOH HOH173-H1
1686 HOH HOH173-H2
1687 HOH HOH174-O
1688 HOH HOH174-H1
1689 HOH HOH174-H2
1690 HOH H

2803 HOH HOH548-O
2804 HOH HOH548-H1
2805 HOH HOH548-H2
2806 HOH HOH549-O
2807 HOH HOH549-H1
2808 HOH HOH549-H2
2809 HOH HOH550-O
2810 HOH HOH550-H1
2811 HOH HOH550-H2
2812 HOH HOH551-O
2813 HOH HOH551-H1
2814 HOH HOH551-H2
2815 HOH HOH552-O
2816 HOH HOH552-H1
2817 HOH HOH552-H2
2818 HOH HOH553-O
2819 HOH HOH553-H1
2820 HOH HOH553-H2
2821 HOH HOH554-O
2822 HOH HOH554-H1
2823 HOH HOH554-H2
2824 HOH HOH555-O
2825 HOH HOH555-H1
2826 HOH HOH555-H2
2827 HOH HOH556-O
2828 HOH HOH556-H1
2829 HOH HOH556-H2
2830 HOH HOH557-O
2831 HOH HOH557-H1
2832 HOH HOH557-H2
2833 HOH HOH558-O
2834 HOH HOH558-H1
2835 HOH HOH558-H2
2836 HOH HOH559-O
2837 HOH HOH559-H1
2838 HOH HOH559-H2
2839 HOH HOH560-O
2840 HOH HOH560-H1
2841 HOH HOH560-H2
2842 HOH HOH561-O
2843 HOH HOH561-H1
2844 HOH HOH561-H2
2845 HOH HOH562-O
2846 HOH HOH562-H1
2847 HOH HOH562-H2
2848 HOH HOH563-O
2849 HOH HOH563-H1
2850 HOH HOH563-H2
2851 HOH HOH564-O
2852 HOH HOH564-H1
2853 HOH HOH564-H2
2854 HOH HOH565-O
2855 HOH HOH565-H1
2856 HOH HO

3970 HOH HOH940-O
3971 HOH HOH940-H1
3972 HOH HOH940-H2
3973 HOH HOH941-O
3974 HOH HOH941-H1
3975 HOH HOH941-H2
3976 HOH HOH942-O
3977 HOH HOH942-H1
3978 HOH HOH942-H2
3979 HOH HOH943-O
3980 HOH HOH943-H1
3981 HOH HOH943-H2
3982 HOH HOH944-O
3983 HOH HOH944-H1
3984 HOH HOH944-H2
3985 HOH HOH945-O
3986 HOH HOH945-H1
3987 HOH HOH945-H2
3988 HOH HOH946-O
3989 HOH HOH946-H1
3990 HOH HOH946-H2
3991 HOH HOH947-O
3992 HOH HOH947-H1
3993 HOH HOH947-H2
3994 HOH HOH948-O
3995 HOH HOH948-H1
3996 HOH HOH948-H2
3997 HOH HOH949-O
3998 HOH HOH949-H1
3999 HOH HOH949-H2
4000 HOH HOH950-O
4001 HOH HOH950-H1
4002 HOH HOH950-H2
4003 HOH HOH951-O
4004 HOH HOH951-H1
4005 HOH HOH951-H2
4006 HOH HOH952-O
4007 HOH HOH952-H1
4008 HOH HOH952-H2
4009 HOH HOH953-O
4010 HOH HOH953-H1
4011 HOH HOH953-H2
4012 HOH HOH954-O
4013 HOH HOH954-H1
4014 HOH HOH954-H2
4015 HOH HOH955-O
4016 HOH HOH955-H1
4017 HOH HOH955-H2
4018 HOH HOH956-O
4019 HOH HOH956-H1
4020 HOH HOH956-H2
4021 HOH HOH957-O
4022 HOH HOH957-H1
4023 HOH HO

...

9969 HOH HOH2943-H2
9970 HOH HOH2944-O
9971 HOH HOH2944-H1
9972 HOH HOH2944-H2
9973 HOH HOH2945-O
9974 HOH HOH2945-H1
9975 HOH HOH2945-H2
9976 HOH HOH2946-O
9977 HOH HOH2946-H1
9978 HOH HOH2946-H2
9979 HOH HOH2947-O
9980 HOH HOH2947-H1
9981 HOH HOH2947-H2
9982 HOH HOH2948-O
9983 HOH HOH2948-H1
9984 HOH HOH2948-H2
9985 HOH HOH2949-O
9986 HOH HOH2949-H1
9987 HOH HOH2949-H2
9988 HOH HOH2950-O
9989 HOH HOH2950-H1
9990 HOH HOH2950-H2
9991 HOH HOH2951-O
9992 HOH HOH2951-H1
9993 HOH HOH2951-H2
9994 HOH HOH2952-O
9995 HOH HOH2952-H1
9996 HOH HOH2952-H2
9997 HOH HOH2953-O
9998 HOH HOH2953-H1
9999 HOH HOH2953-H2
10000 HOH HOH2954-O
10001 HOH HOH2954-H1
10002 HOH HOH2954-H2
10003 HOH HOH2955-O
10004 HOH HOH2955-H1
10005 HOH HOH2955-H2
10006 HOH HOH2956-O
10007 HOH HOH2956-H1
10008 HOH HOH2956-H2
10009 HOH HOH2957-O
10010 HOH HOH2957-H1
10011 HOH HOH2957-H2
10012 HOH HOH2958-O
10013 HOH HOH2958-H1
10014 HOH HOH2958-H2
10015 HOH HOH2959-O
10016 HOH HOH2959-H1
10017 HOH HOH2959-H2
10018 HOH HOH2960-

10969 HOH HOH3279-O
10970 HOH HOH3279-H1
10971 HOH HOH3279-H2
10972 HOH HOH3280-O
10973 HOH HOH3280-H1
10974 HOH HOH3280-H2
10975 HOH HOH3281-O
10976 HOH HOH3281-H1
10977 HOH HOH3281-H2
10978 HOH HOH3282-O
10979 HOH HOH3282-H1
10980 HOH HOH3282-H2
10981 HOH HOH3283-O
10982 HOH HOH3283-H1
10983 HOH HOH3283-H2
10984 HOH HOH3284-O
10985 HOH HOH3284-H1
10986 HOH HOH3284-H2
10987 HOH HOH3285-O
10988 HOH HOH3285-H1
10989 HOH HOH3285-H2
10990 HOH HOH3286-O
10991 HOH HOH3286-H1
10992 HOH HOH3286-H2
10993 HOH HOH3287-O
10994 HOH HOH3287-H1
10995 HOH HOH3287-H2
10996 HOH HOH3288-O
10997 HOH HOH3288-H1
10998 HOH HOH3288-H2
10999 HOH HOH3289-O
11000 HOH HOH3289-H1
11001 HOH HOH3289-H2
11002 HOH HOH3290-O
11003 HOH HOH3290-H1
11004 HOH HOH3290-H2
11005 HOH HOH3291-O
11006 HOH HOH3291-H1
11007 HOH HOH3291-H2
11008 HOH HOH3292-O
11009 HOH HOH3292-H1
11010 HOH HOH3292-H2
11011 HOH HOH3293-O
11012 HOH HOH3293-H1
11013 HOH HOH3293-H2
11014 HOH HOH3294-O
11015 HOH HOH3294-H1
11016 HOH HOH3294-H2
11017 HO

11802 HOH HOH3557-H2
11803 HOH HOH3558-O
11804 HOH HOH3558-H1
11805 HOH HOH3558-H2
11806 HOH HOH3559-O
11807 HOH HOH3559-H1
11808 HOH HOH3559-H2
11809 HOH HOH3560-O
11810 HOH HOH3560-H1
11811 HOH HOH3560-H2
11812 HOH HOH3561-O
11813 HOH HOH3561-H1
11814 HOH HOH3561-H2
11815 HOH HOH3562-O
11816 HOH HOH3562-H1
11817 HOH HOH3562-H2
11818 HOH HOH3563-O
11819 HOH HOH3563-H1
11820 HOH HOH3563-H2
11821 HOH HOH3564-O
11822 HOH HOH3564-H1
11823 HOH HOH3564-H2
11824 HOH HOH3565-O
11825 HOH HOH3565-H1
11826 HOH HOH3565-H2
11827 HOH HOH3566-O
11828 HOH HOH3566-H1
11829 HOH HOH3566-H2
11830 HOH HOH3567-O
11831 HOH HOH3567-H1
11832 HOH HOH3567-H2
11833 HOH HOH3568-O
11834 HOH HOH3568-H1
11835 HOH HOH3568-H2
11836 HOH HOH3569-O
11837 HOH HOH3569-H1
11838 HOH HOH3569-H2
11839 HOH HOH3570-O
11840 HOH HOH3570-H1
11841 HOH HOH3570-H2
11842 HOH HOH3571-O
11843 HOH HOH3571-H1
11844 HOH HOH3571-H2
11845 HOH HOH3572-O
11846 HOH HOH3572-H1
11847 HOH HOH3572-H2
11848 HOH HOH3573-O
11849 HOH HOH3573-H1
11850 HO

13135 HOH HOH4004-O
13136 HOH HOH4004-H1
13137 HOH HOH4004-H2
13138 HOH HOH4005-O
13139 HOH HOH4005-H1
13140 HOH HOH4005-H2
13141 HOH HOH4006-O
13142 HOH HOH4006-H1
13143 HOH HOH4006-H2
13144 HOH HOH4007-O
13145 HOH HOH4007-H1
13146 HOH HOH4007-H2
13147 HOH HOH4008-O
13148 HOH HOH4008-H1
13149 HOH HOH4008-H2
13150 HOH HOH4009-O
13151 HOH HOH4009-H1
13152 HOH HOH4009-H2
13153 HOH HOH4010-O
13154 HOH HOH4010-H1
13155 HOH HOH4010-H2
13156 HOH HOH4011-O
13157 HOH HOH4011-H1
13158 HOH HOH4011-H2
13159 HOH HOH4012-O
13160 HOH HOH4012-H1
13161 HOH HOH4012-H2
13162 HOH HOH4013-O
13163 HOH HOH4013-H1
13164 HOH HOH4013-H2
13165 HOH HOH4014-O
13166 HOH HOH4014-H1
13167 HOH HOH4014-H2
13168 HOH HOH4015-O
13169 HOH HOH4015-H1
13170 HOH HOH4015-H2
13171 HOH HOH4016-O
13172 HOH HOH4016-H1
13173 HOH HOH4016-H2
13174 HOH HOH4017-O
13175 HOH HOH4017-H1
13176 HOH HOH4017-H2
13177 HOH HOH4018-O
13178 HOH HOH4018-H1
13179 HOH HOH4018-H2
13180 HOH HOH4019-O
13181 HOH HOH4019-H1
13182 HOH HOH4019-H2
13183 HO

14301 HOH HOH4392-H2
14302 HOH HOH4393-O
14303 HOH HOH4393-H1
14304 HOH HOH4393-H2
14305 HOH HOH4394-O
14306 HOH HOH4394-H1
14307 HOH HOH4394-H2
14308 HOH HOH4395-O
14309 HOH HOH4395-H1
14310 HOH HOH4395-H2
14311 HOH HOH4396-O
14312 HOH HOH4396-H1
14313 HOH HOH4396-H2
14314 HOH HOH4397-O
14315 HOH HOH4397-H1
14316 HOH HOH4397-H2
14317 HOH HOH4398-O
14318 HOH HOH4398-H1
14319 HOH HOH4398-H2
14320 HOH HOH4399-O
14321 HOH HOH4399-H1
14322 HOH HOH4399-H2
14323 HOH HOH4400-O
14324 HOH HOH4400-H1
14325 HOH HOH4400-H2
14326 HOH HOH4401-O
14327 HOH HOH4401-H1
14328 HOH HOH4401-H2
14329 HOH HOH4402-O
14330 HOH HOH4402-H1
14331 HOH HOH4402-H2
14332 HOH HOH4403-O
14333 HOH HOH4403-H1
14334 HOH HOH4403-H2
14335 HOH HOH4404-O
14336 HOH HOH4404-H1
14337 HOH HOH4404-H2
14338 HOH HOH4405-O
14339 HOH HOH4405-H1
14340 HOH HOH4405-H2
14341 HOH HOH4406-O
14342 HOH HOH4406-H1
14343 HOH HOH4406-H2
14344 HOH HOH4407-O
14345 HOH HOH4407-H1
14346 HOH HOH4407-H2
14347 HOH HOH4408-O
14348 HOH HOH4408-H1
14349 HO

In [6]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)


In [7]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)

In [62]:
cycle_start = time.time()
# Run neq
ncycles = 1
forward_works_master, reverse_works_master = list(), list()
forward_eq_old, forward_neq_old, forward_neq_new = list(), list(), list()
reverse_eq_new, reverse_neq_old, reverse_neq_new = list(), list(), list()
for cycle in range(ncycles):
    # Equilibrium (lambda = 0)
    for step in range(nsteps_eq):
        _logger.info(f'Cycle: {cycle}, Starting to equilibrate at lambda = 0')
        initial_time = time.time()
        integrator.step(1)
        elapsed_time = (time.time() - initial_time) * unit.seconds
        _logger.info(f'Cycle: {cycle}, Step: {step}, equilibrating at lambda = 0, took: {elapsed_time / unit.seconds} seconds')
#         if step % 250 == 0:
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        old_pos = np.asarray(htf.old_positions(pos))
        traj = md.Trajectory(old_pos, md.Topology.from_openmm(htf._topology_proposal.old_topology))
        old_pos = traj.atom_slice(traj.top.select("not water")).xyz[0]
        forward_eq_old.append(old_pos)

    # Forward (0 -> 1)
    forward_works = [integrator.get_protocol_work(dimensionless=True)]
    for fwd_step in range(nsteps_neq):
        initial_time = time.time()
        integrator.step(1)
        elapsed_time = (time.time() - initial_time) * unit.seconds
        _logger.info(f'Cycle: {cycle}, forward NEQ step: {fwd_step}, took: {elapsed_time / unit.seconds} seconds')
        forward_works.append(integrator.get_protocol_work(dimensionless=True))
        if fwd_step % 250 == 0:
            pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
            old_pos = np.asarray(htf.old_positions(pos))
            new_pos = np.asarray(htf.new_positions(pos))
            forward_neq_old.append(old_pos)
            forward_neq_new.append(new_pos)
    forward_works_master.append(forward_works)
    
    # Equilibrium (lambda = 1)
    for step in range(nsteps_eq):
        _logger.info(f'Cycle: {cycle}, Starting to equilibrate at lambda = 1')
        initial_time = time.time()
        integrator.step(1)
        elapsed_time = (time.time() - initial_time) * unit.seconds
        _logger.info(f'Cycle: {cycle}, Step: {step}, equilibrating at lambda = 1, took: {elapsed_time / unit.seconds} seconds')
#         if step % 250 == 0:
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        new_pos = np.asarray(htf.new_positions(pos))
        reverse_eq_new.append(new_pos)

    # Reverse work (1 -> 0)
    reverse_works = [integrator.get_protocol_work(dimensionless=True)]
    for rev_step in range(nsteps_neq):
        initial_time = time.time()
        integrator.step(1)
        elapsed_time = (time.time() - initial_time) * unit.seconds
        _logger.info(f'Cycle: {cycle}, reverse NEQ step: {rev_step}, took: {elapsed_time / unit.seconds} seconds')
        reverse_works.append(integrator.get_protocol_work(dimensionless=True))
        if rev_step % 250 == 0:
            pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
            old_pos = np.asarray(htf.old_positions(pos))
            new_pos = np.asarray(htf.new_positions(pos))
            reverse_neq_old.append(old_pos)
            reverse_neq_new.append(new_pos)
    reverse_works_master.append(reverse_works)
print(f"cycle took: {(time.time() - cycle_start) * unit.seconds}")


INFO:root:Cycle: 0, Starting to equilibrate at lambda = 0
INFO:root:Cycle: 0, Step: 0, equilibrating at lambda = 0, took: 0.002431631088256836 seconds
INFO:root:Cycle: 0, Starting to equilibrate at lambda = 0
INFO:root:Cycle: 0, Step: 1, equilibrating at lambda = 0, took: 0.0027844905853271484 seconds
INFO:root:Cycle: 0, forward NEQ step: 0, took: 0.002353191375732422 seconds
INFO:root:Cycle: 0, Starting to equilibrate at lambda = 1
INFO:root:Cycle: 0, Step: 0, equilibrating at lambda = 1, took: 0.0019025802612304688 seconds
INFO:root:Cycle: 0, Starting to equilibrate at lambda = 1
INFO:root:Cycle: 0, Step: 1, equilibrating at lambda = 1, took: 0.0020599365234375 seconds
INFO:root:Cycle: 0, reverse NEQ step: 0, took: 0.002416849136352539 seconds


cycle took: 5.121992826461792 s


In [13]:
# # Save works
# with open("0_forward.npy", 'wb') as f:
#     np.save(f, forward_works_master)
# with open("0_reverse.npy", 'wb') as f:
#     np.save(f, reverse_works_master)

# Save trajs
top_old = md.Topology.from_openmm(htf._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htf._topology_proposal.new_topology)
# traj = md.Trajectory(np.array(forward_eq_old), top_old)
# traj.remove_solvent(inplace=True)
# traj.save("0_forward_eq_old.pdb")
# traj = md.Trajectory(np.array(reverse_eq_new), top_new)
# traj.remove_solvent(inplace=True)
# traj.save("0_reverse_eq_new.pdb")

# traj = md.Trajectory(np.array(forward_neq_old), top_old)
# traj.remove_solvent(inplace=True)
# traj.save("0_forward_neq_old.pdb")
# traj = md.Trajectory(np.array(forward_neq_new), top_new)
# traj.remove_solvent(inplace=True)
# traj.save("0_forward_neq_new.pdb")

# traj = md.Trajectory(np.array(reverse_neq_old), top_old)
# traj.remove_solvent(inplace=True)
# traj.save("0_reverse_neq_old.pdb")
traj = md.Trajectory(np.array(reverse_neq_new), top_new)
traj.remove_solvent(inplace=True)
traj.save("0_reverse_neq_new.pdb")


In [63]:
forward_eq_old = np.array(forward_eq_old)

In [64]:
forward_eq_old.shape

(2, 1455, 3)

In [65]:
np.array(reverse_eq_new).shape

(2, 14873, 3)

In [66]:
forward_eq_old[0].shape

(1455, 3)

In [67]:
old_pos = traj.atom_slice(traj.top.select("not water")).xyz

In [68]:
old_pos[0].shape

(1455, 3)